In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="7WDcRR7o9Dze5mRQltRg")
project = rf.workspace("dream-vj9o9").project("tobacco-segmentation-yn65w")
version = project.version(4)
dataset = version.download("yolov8")
                

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Define file paths
image_path = "/kaggle/working/tobacco-segmentation-4/train/images/tobacco_0251_jpg.rf.33207206762f99d9705271df0b468791.jpg"
label_path = "/kaggle/working/tobacco-segmentation-4/train/labels/tobacco_0251_jpg.rf.33207206762f99d9705271df0b468791.txt"

# Read the image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

# Read the label file
with open(label_path, "r") as f:
    lines = f.readlines()

height, width, _ = image.shape

# Parse segmentation coordinates
for line in lines:
    parts = line.strip().split()
    class_id = int(parts[0])  # First value is the class ID
    points = np.array(parts[1:], dtype=np.float32).reshape(-1, 2)

    # Convert normalized coordinates to image dimensions
    points[:, 0] *= width  # Scale X coordinates
    points[:, 1] *= height  # Scale Y coordinates

    # Convert to integer format for drawing
    points = points.astype(np.int32)

    # Draw segmentation
    cv2.polylines(image, [points], isClosed=True, color=(255, 0, 0), thickness=2)
    # cv2.fillPoly(image, [points], color=(0, 255, 0))  # Transparent fill

# Show the image with segmentation
plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.axis("off")
plt.show()


In [ ]:
import os

# Define the content of data.yaml
yaml_content = """names:
  - tobacco
nc: 1
train: ../train/images
val: ../train/images  # Using train set for validation since valid set is missing
"""

# Define the path to save data.yaml
yaml_path = "/kaggle/working/tobacco-segmentation-4/data.yaml"

# Create necessary directories if they don't exist
os.makedirs(os.path.dirname(yaml_path), exist_ok=True)

# Write data.yaml file
with open(yaml_path, "w") as file:
    file.write(yaml_content)

print(f"✅ data.yaml created at {yaml_path}")




In [ ]:
from ultralytics import YOLO
import torch

# Free up GPU memory
torch.cuda.empty_cache()

# Use a smaller model to avoid OOM errors
model = YOLO("yolov8l-seg.pt")

# Train the model with optimized settings and augmentations
model.train(
    data="/kaggle/working/tobacco-segmentation-4/data.yaml",
    epochs=1000,
    batch=2,  # 🔥 Lower batch size (try 1 if needed)
    imgsz=640,
    workers=2,
    device="cuda",

    # Augmentations for better generalization
    hsv_h=0.015,  # Random hue shift
    hsv_s=0.7,  # Random saturation shift
    hsv_v=0.4,  # Random brightness shift
    flipud=0.5,  # Random vertical flip
    fliplr=0.5,  # Random horizontal flip
    mosaic=1.0,  # Use mosaic augmentation
    mixup=0.2,  # Mixup augmentation for blending images
    degrees=10,  # Random rotation
    translate=0.1,  # Random translation
    scale=0.5,  # Random scaling
    shear=2,  # Random shearing
)

print("✅ Training started successfully with augmentations!")


In [ ]:
from ultralytics import YOLO

# Load the trained model
model = YOLO("/kaggle/working/runs/segment/train3/weights/best.pt")  

# Run validation
results = model.val()

# Print mAP score (mAP50 and mAP50-95)
print(f"📊 mAP@50: {results.box.map50:.3f}")  # mAP@50
print(f"📊 mAP@50-95: {results.box.map:.3f}")  # mAP@50-95


In [ ]:
from ultralytics import YOLO

# Load the trained YOLOv8 segmentation model
model = YOLO("/kaggle/working/runs/segment/train3/weights/last.pt")  

# Run validation
results = model.val()

# Extract and print segmentation evaluation metrics
print(f"📊 Precision: {results.seg.p.mean():.3f}")  # Mean Precision
print(f"📊 Recall: {results.seg.r.mean():.3f}")  # Mean Recall

# Show detailed results
print("\n🔍 Full results dictionary:")
print(results.results_dict)
